<a href="https://colab.research.google.com/github/JacekPardyak/vps/blob/master/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [19]:
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [26]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [27]:
!pip install -q findspark
import findspark
findspark.init()

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [29]:
!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2021-09-04 19:24:43--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2021-09-04 19:24:43 (18.7 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]



In [63]:
!wget --continue https://raw.githubusercontent.com/JacekPardyak/vps/master/data/vps_churn_data_py.json -O /tmp/vps_churn_data_py.json

--2021-09-04 20:36:12--  https://raw.githubusercontent.com/JacekPardyak/vps/master/data/vps_churn_data_py.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247125 (241K) [text/plain]
Saving to: ‘/tmp/vps_churn_data_py.json’

/tmp/vps_churn_data 100%[===================>] 241.33K  --.-KB/s    in 0.06s   

2021-09-04 20:36:12 (3.99 MB/s) - ‘/tmp/vps_churn_data_py.json’ saved [247125/247125]



In [64]:
#df = spark.read.json("/tmp/sample_books.json")
df = spark.read.json("/tmp/vps_churn_data_py.json")

In [65]:
df.printSchema()

root
 |-- cpu_load_max_gradient: double (nullable = true)
 |-- cpu_load_mean_m_3: double (nullable = true)
 |-- cpu_load_monthly_mean_delta: double (nullable = true)
 |-- disk_octets_read_max_gradient: double (nullable = true)
 |-- disk_octets_read_mean_m_3: double (nullable = true)
 |-- disk_octets_read_monthly_mean_delta: double (nullable = true)
 |-- disk_octets_write_max_gradient: double (nullable = true)
 |-- disk_octets_write_mean_m_3: double (nullable = true)
 |-- disk_octets_write_monthly_mean_delta: double (nullable = true)
 |-- disk_ops_read_max_gradient: double (nullable = true)
 |-- disk_ops_read_mean_m_3: double (nullable = true)
 |-- disk_ops_read_monthly_mean_delta: double (nullable = true)
 |-- disk_ops_write_max_gradient: double (nullable = true)
 |-- disk_ops_write_mean_m_3: double (nullable = true)
 |-- disk_ops_write_monthly_mean_delta: double (nullable = true)
 |-- id: long (nullable = true)
 |-- is_churn: long (nullable = true)
 |-- network_rx_max_gradient: double

In [66]:
df.show(4,False)

+---------------------+-----------------+---------------------------+-----------------------------+-------------------------+-----------------------------------+------------------------------+--------------------------+------------------------------------+--------------------------+----------------------+--------------------------------+---------------------------+-----------------------+---------------------------------+---+--------+-----------------------+-------------------+-----------------------------+-----------------------+-------------------+-----------------------------+
|cpu_load_max_gradient|cpu_load_mean_m_3|cpu_load_monthly_mean_delta|disk_octets_read_max_gradient|disk_octets_read_mean_m_3|disk_octets_read_monthly_mean_delta|disk_octets_write_max_gradient|disk_octets_write_mean_m_3|disk_octets_write_monthly_mean_delta|disk_ops_read_max_gradient|disk_ops_read_mean_m_3|disk_ops_read_monthly_mean_delta|disk_ops_write_max_gradient|disk_ops_write_mean_m_3|disk_ops_write_monthly

In [67]:
df.count()

283

In [68]:
df.select('id', 'cpu_load_max_gradient', 'is_churn').show(5)

+---+---------------------+--------+
| id|cpu_load_max_gradient|is_churn|
+---+---------------------+--------+
|100|           0.44565168|       1|
|101|          -0.00172695|       0|
|102|            0.3733529|       0|
|103|          -0.03158981|       1|
|104|          -0.09440948|       0|
+---+---------------------+--------+
only showing top 5 rows



In [51]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50, False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [52]:
df_filtered.select("title", "year_written").filter("title LIKE '%Harry Potter%'").distinct().show(20, False)

+------------+------------+
|title       |year_written|
+------------+------------+
|Harry Potter|2000        |
+------------+------------+



In [53]:
from pyspark.sql.functions import max
# Find the costliest book
maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+

